In [38]:
!ls /workspace/results/syn_v0/

end2end_quartznet_export_dev.onnx
end2end_quartznet_folded.onnx
end2end_quartznet_folded_renamed.onnx
end2end_quartznet_folded_test.onnx
end2end_quartznet_hls_layers.onnx
end2end_quartznet_lowered.onnx
end2end_quartznet_lowered_partitioned.onnx
end2end_quartznet_streamline.onnx
end2end_quartznet_tidy.onnx
experiment_partitions
partitioning_repartition


# Copy all files to new directory for testing

In [50]:
!mkdir /workspace/results/syn_v0/experiment_partitions
!cp /workspace/results/syn_v0/end2end_quartznet_folded.onnx /workspace/results/syn_v0/experiment_partitions/
!cp -r /workspace/results/syn_v0/partitioning_repartition/. /workspace/results/syn_v0/experiment_partitions/

mkdir: cannot create directory '/workspace/results/syn_v0/experiment_partitions': File exists


In [71]:
!cp /workspace/results/syn_v0/end2end_quartznet_folded.onnx /workspace/results/syn_v0/experiment_partitions/
!cp -r /workspace/results/syn_v0/partitioning_repartition/. /workspace/results/syn_v0/experiment_partitions/

In [35]:
!ls /workspace/results/syn_v0/experiment_partitions/

end2end_quartznet_folded.onnx			partition_16.onnx
end2end_quartznet_folded_cropped.onnx		partition_1_cropped.onnx
end2end_quartznet_folded_cropped_extended.onnx	partition_2.onnx
partition_0.onnx				partition_2_cropped.onnx
partition_1.onnx				partition_3.onnx
partition_10.onnx				partition_4.onnx
partition_10_cropped.onnx			partition_4_cropped.onnx
partition_11.onnx				partition_5.onnx
partition_12.onnx				partition_6.onnx
partition_13.onnx				partition_7.onnx
partition_13_cropped.onnx			partition_7_cropped.onnx
partition_14.onnx				partition_8.onnx
partition_15.onnx				partition_9.onnx


# Rename model path (if needed..)

In [72]:
from finn.custom_op.registry import getCustomOp
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

for n in model.graph.node:
    if n.op_type=="GenericPartition":
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        new_model_path = model_path.replace("/finn_dev_mirza/partitioning_repartition", "/syn_v0/experiment_partitions")
        inst.set_nodeattr("model", new_model_path)
                
model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

In [74]:
showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx' at http://0.0.0.0:8081


# Repartition graph (exclude Transpose node in first partition)
## CAUTION: run only once...

In [75]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.extend_partition import ExtendPartition
from finn.transformation.create_generic_partitions import PartitionFromDict

partitioning = {0: range(1,4)}

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

model = model.transform(ExtendPartition([0]))
model = model.transform(PartitionFromDict(partitioning, "/workspace/results/syn_v0/experiment_partitions"))

model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")
#showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

In [76]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx' at http://0.0.0.0:8081


# SetFolding

In [80]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.fpgadataflow.set_folding import SetFolding

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

for n in model.graph.node:
    if n.op_type=="GenericPartition":
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        model_partition = ModelWrapper(model_path)
        model_partition = model_partition.transform(SetFolding(target_cycles_per_frame=1200000))  
        model_partition.save(model_path)
        
#model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

In [105]:
## COUNT nodes that have certain PE, SIMD value

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")
pe_dict={}
for n in model.graph.node:
    if n.op_type=="GenericPartition":
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        model_partition = ModelWrapper(model_path)
        for npar in model_partition.graph.node:
            inst_par = getCustomOp(npar)
            if npar.op_type=="ConvolutionInputGenerator1D":
                pe = inst_par.get_nodeattr("SIMD")
                if pe in pe_dict:
                    pe_dict[pe] += 1
                else:
                    pe_dict[pe] = 1
        

In [99]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_v0/experiment_partitions/partition_4.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/partition_4.onnx' at http://0.0.0.0:8081


# Crop model

In [135]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.general import (
    RemoveStaticGraphInputs,
    RemoveUnusedTensors,
    SortGraph,
)
from finn.transformation.infer_shapes import InferShapes

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_1.onnx")
#for idx,n in enumerate(model.graph.node):
#    print("{}\t{}".format(idx,n.name))

idx_to_remove = [7,8,9,10,11,12,13,14,15,16,17,18]
nodes = [n for n in model.graph.node]
for idx, n in enumerate(nodes):
    if n.name=="pt_1_FMPadding_Batch_1":
        inp_connect = n.input[0]
    if n.name=="pt_1_FMPadding_Batch_4":
        n.input[0]=inp_connect
    if idx in idx_to_remove:
        model.graph.node.remove(n) 

model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph())        
#model = model.transform(InferShapes())
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_1.onnx")
############################################################################################################

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_4.onnx")
idx_to_remove = [2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
nodes = [n for n in model.graph.node]
for idx, n in enumerate(nodes):
    if n.name=="pt_4_FMPadding_Batch_0":
        inp_connect = n.input[0]
    if n.name=="pt_4_FMPadding_Batch_4":
        n.input[0]=inp_connect
    if idx in idx_to_remove:
        model.graph.node.remove(n) 

model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph())        
#model = model.transform(InferShapes())
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_4.onnx")
############################################################################################################

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_7.onnx")
idx_to_remove = [7,8,9,10,11,12,13,14,15,16,17,18]
nodes = [n for n in model.graph.node]
for idx, n in enumerate(nodes):
    if n.name=="pt_7_FMPadding_Batch_1":
        inp_connect = n.input[0]
    if n.name=="pt_7_FMPadding_Batch_4":
        n.input[0]=inp_connect
    if idx in idx_to_remove:
        model.graph.node.remove(n) 

model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph())        
#model = model.transform(InferShapes())
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_7.onnx")
############################################################################################################

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_10.onnx")
idx_to_remove = [2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
nodes = [n for n in model.graph.node]
for idx, n in enumerate(nodes):
    if n.name=="pt_10_FMPadding_Batch_0":
        inp_connect = n.input[0]
    if n.name=="pt_10_FMPadding_Batch_4":
        n.input[0]=inp_connect
    if idx in idx_to_remove:
        model.graph.node.remove(n) 

model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph())        
#model = model.transform(InferShapes())
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_10.onnx")
############################################################################################################

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_13.onnx")
idx_to_remove = [2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
nodes = [n for n in model.graph.node]
for idx, n in enumerate(nodes):
    if n.name=="pt_13_FMPadding_Batch_0":
        inp_connect = n.input[0]
    if n.name=="pt_13_FMPadding_Batch_4":
        n.input[0]=inp_connect
    if idx in idx_to_remove:
        model.graph.node.remove(n) 

model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph())        
#model = model.transform(InferShapes())
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_13.onnx")
############################################################################################################

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/partition_16.onnx")
model.save("/workspace/results/syn_v0/experiment_partitions/cropped_partition_16.onnx")

In [86]:
!ls /workspace/results/syn_v0/experiment_partitions/

cropped_partition_1.onnx       partition_10.onnx  partition_3.onnx
cropped_partition_10.onnx      partition_11.onnx  partition_4.onnx
cropped_partition_13.onnx      partition_12.onnx  partition_5.onnx
cropped_partition_4.onnx       partition_13.onnx  partition_6.onnx
cropped_partition_7.onnx       partition_14.onnx  partition_7.onnx
end2end_quartznet_folded.onnx  partition_15.onnx  partition_8.onnx
partition_0.onnx	       partition_16.onnx  partition_9.onnx
partition_1.onnx	       partition_2.onnx


In [138]:
showInNetron("/workspace/results/syn_v0/experiment_partitions/cropped_partition_10.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/cropped_partition_10.onnx' at http://0.0.0.0:8081


In [137]:
showInNetron("/workspace/results/syn_v0/experiment_partitions/partition_10.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/partition_10.onnx' at http://0.0.0.0:8081


## Construct new model out of cropped ones

In [108]:
!ls /workspace/results/syn_v0/experiment_partitions/

cropped_partition_1.onnx       partition_10.onnx  partition_3.onnx
cropped_partition_10.onnx      partition_11.onnx  partition_4.onnx
cropped_partition_13.onnx      partition_12.onnx  partition_5.onnx
cropped_partition_4.onnx       partition_13.onnx  partition_6.onnx
cropped_partition_7.onnx       partition_14.onnx  partition_7.onnx
end2end_quartznet_folded.onnx  partition_15.onnx  partition_8.onnx
partition_0.onnx	       partition_16.onnx  partition_9.onnx
partition_1.onnx	       partition_2.onnx


In [139]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.general import (
    RemoveStaticGraphInputs,
    RemoveUnusedTensors,
    SortGraph,
)
from finn.custom_op.registry import getCustomOp

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded.onnx")

def reshape_tensors_and_attributes_partition(n, model, outp_connect):
        outp_connect_vi = model.get_tensor_valueinfo(outp_connect)
        print(outp_connect)
        # Change input within GenericPartition graph
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        new_model_path = model_path.replace("/partition_", "/cropped_partition_")
        inst.set_nodeattr("model", new_model_path)
        model_partition = ModelWrapper(new_model_path)
        for m in model_partition.graph.input:
            print(m.name)
            cons = model_partition.find_consumer(m.name)
            cons.input[0] = outp_connect_vi.name
        model_partition.graph.input.remove(model_partition.graph.input[0])
        model_partition.graph.input.insert(0, outp_connect_vi)
        model_partition.save(new_model_path)

nodes_to_keep = ["GenericPartition_0", "GenericPartition_1", "GenericPartition_4", "GenericPartition_7", "GenericPartition_10", "GenericPartition_13", "GenericPartition_16"]
for n in model.graph.node:
    if n.name=="GenericPartition_0":
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_1":
        n.input[0] = outp_connect
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        new_model_path = model_path.replace("/partition_", "/cropped_partition_")
        inst.set_nodeattr("model", new_model_path)
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_2":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_3":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_4":
        # Change input of GenericPartition
        n.input[0] = outp_connect
        reshape_tensors_and_attributes_partition(n, model, outp_connect)
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_5":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_6":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_7":
        # Change input of GenericPartition
        n.input[0] = outp_connect
        reshape_tensors_and_attributes_partition(n, model, outp_connect)
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_8":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_9":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_10":
        n.input[0] = outp_connect
        reshape_tensors_and_attributes_partition(n, model, outp_connect)
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_11":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_12":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_13":
        n.input[0] = outp_connect
        reshape_tensors_and_attributes_partition(n, model, outp_connect)
        outp_connect = n.output[0]
    elif n.name=="GenericPartition_14":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_15":
        model.graph.node.remove(n)
    elif n.name=="GenericPartition_16":
        n.input[0] = outp_connect
        reshape_tensors_and_attributes_partition(n, model, outp_connect)
        
model = model.transform(RemoveUnusedTensors())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(SortGraph()) 

model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped.onnx")

MultiThreshold_12_out0
MultiThreshold_36_out0
MultiThreshold_48_out0
MultiThreshold_72_out0
MultiThreshold_84_out0
MultiThreshold_108_out0
MultiThreshold_120_out0
MultiThreshold_144_out0
MultiThreshold_156_out0
MultiThreshold_180_out0


In [140]:
showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped.onnx' at http://0.0.0.0:8081


## Extend partitions

In [149]:
from finn.transformation.extend_partition import ExtendPartition

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped.onnx")

for idx,n in enumerate(model.graph.node):
    if n.op_type=="GenericPartition":
        print(idx)
        
model = model.transform(ExtendPartition([1,2,3,4,5,6,7]))

model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended.onnx")

1
2
3
4
5
6
7


In [150]:
showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended.onnx' at http://0.0.0.0:8081


# Clean up node attributes (caused by previous synthesis)

In [152]:
# Verify InferShapes works... (it does!)
from finn.transformation.infer_shapes import InferShapes
model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended.onnx")
model = model.transform(InferShapes())

In [23]:
from finn.custom_op.registry import getCustomOp
from finn.core.modelwrapper import ModelWrapper
from finn.util.fpgadataflow import is_fpgadataflow_node

model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended.onnx")

for n in model.graph.node:
    if is_fpgadataflow_node(n):        
        attribute_names = [a for a in n.attribute]
        for a in attribute_names:
            if a.name=="code_gen_dir_ipgen": # PrepareIP
                n.attribute.remove(a)
            elif a.name=="ipgen_path": # HLSSynthIP
                n.attribute.remove(a)
            elif a.name=="ip_path": # HLSSynthIP
                n.attribute.remove(a)
            elif a.name=="ip_vlnv": # HLSSynthIP
                n.attribute.remove(a)
            elif a.name=="res_hls": # AnnotateResources
                n.attribute.remove(a)
            elif a.name=="estimated_clk_hls": # Own analysis
                n.attribute.remove(a)
            elif a.name=="est_clk_hls": # Own analysis
                n.attribute.remove(a)
            elif a.name=="est_latency_hls": # Own analysis
                n.attribute.remove(a)
            else:
                continue
        
model.save("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended_cleaned.onnx")
    

In [25]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended_cleaned.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended_cleaned.onnx' at http://0.0.0.0:8081


# Set Folding + type (ONLY)

In [1]:
!ls /workspace/results/syn_v0/experiment_partitions/

cropped_partition_1.onnx				partition_10.onnx
cropped_partition_10.onnx				partition_11.onnx
cropped_partition_13.onnx				partition_12.onnx
cropped_partition_16.onnx				partition_13.onnx
cropped_partition_4.onnx				partition_14.onnx
cropped_partition_7.onnx				partition_15.onnx
end2end_quartznet_folded.onnx				partition_16.onnx
end2end_quartznet_folded_cropped.onnx			partition_2.onnx
end2end_quartznet_folded_cropped_extended.onnx		partition_3.onnx
end2end_quartznet_folded_cropped_extended_cleaned.onnx	partition_4.onnx
end2end_quartznet_syn_v2.onnx				partition_5.onnx
end2end_quartznet_syn_v3.onnx				partition_6.onnx
end2end_quartznet_syn_v4.onnx				partition_7.onnx
partition_0.onnx					partition_8.onnx
partition_1.onnx					partition_9.onnx


In [1]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.transformation.fpgadataflow.set_folding import SetFolding


###########################################################################################
file_path = "/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_syn_v5.onnx"
###########################################################################################



model = ModelWrapper("/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_folded_cropped_extended_cleaned.onnx")

## TODO: create fpgadataflow partition and then run SetFolding..
# now, we are just saving the end2end_quartznet_folded_cropped_extended_cleaned to file_path
for n in model.graph.node:
    if n.op_type=="GenericPartition":
        inst = getCustomOp(n)
        model_path = inst.get_nodeattr("model")
        model_partition = ModelWrapper(model_path)
        model_partition = model_partition.transform(SetFolding(target_cycles_per_frame=1200000))  
        model_partition.save(model_path)
        
model.save(file_path)

In [2]:
from finn.util.visualization import showInNetron
showInNetron(file_path)

Serving '/workspace/results/syn_v0/end2end_quartznet_folded_renamed_extended.onnx' at http://0.0.0.0:8081


In [2]:
from finn.custom_op.registry import getCustomOp

for n in model.graph.node:
    if n.name=="pt_4_Thresholding_Batch_0":
        inst = getCustomOp(n)
        #PE = inst.get_nodeattr("PE")
        inst.set_nodeattr("PE", 4)
    elif n.name=="pt_7_Thresholding_Batch_0":
        inst = getCustomOp(n)
        #PE = inst.get_nodeattr("PE")
        inst.set_nodeattr("PE", 4)
    elif n.name=="pt_10_Thresholding_Batch_0":
        inst = getCustomOp(n)
        #PE = inst.get_nodeattr("PE")
        inst.set_nodeattr("PE", 8)
    elif n.name=="pt_13_Thresholding_Batch_0":
        inst = getCustomOp(n)
        #PE = inst.get_nodeattr("PE")
        inst.set_nodeattr("PE", 8)
    elif n.name=="pt_16_Thresholding_Batch_0":
        inst = getCustomOp(n)
        #PE = inst.get_nodeattr("PE")
        inst.set_nodeattr("PE", 8)
    else:
        continue
        
model.save(file_path)

In [3]:
from finn.util.visualization import showInNetron
showInNetron(file_path)

Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_syn_v5.onnx' at http://0.0.0.0:8081


In [8]:
from finn.core.modelwrapper import ModelWrapper
from finn.util.fpgadataflow import is_fpgadataflow_node
from finn.custom_op.registry import getCustomOp

model = ModelWrapper(file_path)
#model = ModelWrapper("/workspace/results/syn_v0/end2end_quartznet_folded_renamed_extended.onnx")

for n in model.graph.node:
    if is_fpgadataflow_node(n):
        inst = getCustomOp(n)
        if n.op_type=="FMPadding_Batch":
            continue
        elif n.op_type=="DuplicateStreams_Batch":
            continue
        elif n.op_type=="AddStreams_Batch":
            continue
            
        elif n.op_type=="ConvolutionInputGenerator1D":
            inst.set_nodeattr("ram_style", "distributed")
            
        elif n.op_type=="Vector_Vector_Activate_Batch":
            inst.set_nodeattr("resType", "dsp")
            
        elif n.op_type=="StreamingFCLayer_Batch":
            inst.set_nodeattr("resType", "dsp")
            inst.set_nodeattr("ram_style", "ultra")
            inst.set_nodeattr("mem_mode", "decoupled")
            if inst.get_nodeattr("ram_style")=="ultra":
                inst.set_nodeattr("runtime_writeable_weights", 1)
            
        elif n.op_type=="Thresholding_Batch":
            inst.set_nodeattr("ram_style", "distributed")
            inst.set_nodeattr("mem_mode", "decoupled")
            
        else:
            print("Missed: {}".format(n.op_type))
            break


model.save(file_path)

# Verify new syn model visually

In [5]:
from finn.util.visualization import showInNetron
showInNetron(file_path)

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v0/experiment_partitions/end2end_quartznet_syn_v5.onnx' at http://0.0.0.0:8081


In [6]:
showInNetron("/workspace/results/syn_v5/end2end_quartznet_syn_v5.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v5/end2end_quartznet_syn_v5.onnx' at http://0.0.0.0:8081


In [17]:
!ls /workspace/results/

syn_v0	syn_v1	syn_v2	syn_v3	syn_v4	syn_v5
